In [5]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO(r'F:\Data Science\Egg detection with count\egg_detection_with_counts\yolov8n_best_trained_model.pt')

# Initialize video capture
cap = cv2.VideoCapture(r'F:\Data Science\Egg detection with count\egg_detection_with_counts\Eggs_On_A_Conveyor_preview.mp4')

# Define reference line coordinates (for counting crossing)
line_position = 700  # Y-coordinate of the reference line (adjust based on video)
line_color = (0, 255, 0)  # Green line color
line_thickness = 2

# Initialize variables
object_counter = 0  # To count objects crossing the line
tracked_objects = {}  # To store tracked objects and their positions
object_id = 0  # Unique object ID for tracking

# Function to check if the object has crossed the reference line
def has_crossed_line(prev_y, curr_y, line_y):
    return prev_y < line_y and curr_y >= line_y

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection using YOLOv8
    results = model(frame)

    # Get detected object details
    detections = results[0].boxes.xyxy.cpu().numpy()  # Get bounding box coordinates
    confidences = results[0].boxes.conf.cpu().numpy()  # Get confidence scores

    # Draw the reference line
    cv2.line(frame, (0, line_position), (frame.shape[1], line_position), line_color, line_thickness)

    # Track objects and count when they cross the line
    new_tracked_objects = {}

    for i, (x1, y1, x2, y2) in enumerate(detections):
        conf = confidences[i]
        if conf > 0.5:  # Filter based on confidence
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            # Track object positions
            tracked = False
            for obj_id, prev_pos in tracked_objects.items():
                prev_x, prev_y = prev_pos

                # Check if the object is the same by proximity
                if abs(center_x - prev_x) < 50 and abs(center_y - prev_y) < 50:
                    new_tracked_objects[obj_id] = (center_x, center_y)

                    # Check if the object crossed the line
                    if has_crossed_line(prev_y, center_y, line_position):
                        object_counter += 1
                        print(f"Object {obj_id} crossed the line. Total count: {object_counter}")
                    tracked = True
                    break

            # If the object is new, assign it a new ID
            if not tracked:
                object_id += 1
                new_tracked_objects[object_id] = (center_x, center_y)

            # Draw bounding box and center point
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)  # Blue box
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)  # Red center point

    # Update tracked objects for the next frame
    tracked_objects = new_tracked_objects

    # Display the count on the frame
    cv2.putText(frame, f"Count: {object_counter}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the video frame with detection
    cv2.imshow('YOLOv8 Object Counting', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()



0: 384x640 64 eggs, 80.7ms
Speed: 4.0ms preprocess, 80.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 62 eggs, 22.0ms
Speed: 4.0ms preprocess, 22.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 67 eggs, 21.0ms
Speed: 3.0ms preprocess, 21.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 65 eggs, 16.0ms
Speed: 4.0ms preprocess, 16.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 64 eggs, 15.0ms
Speed: 3.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 64 eggs, 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 66 eggs, 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 68 eggs, 16.0ms
Speed: 3.0ms preprocess, 16.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


In [2]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import time

# Load YOLOv8 model
model = YOLO(r'F:\Data Science\Egg detection with count\egg_detection_with_counts\yolov8n_best_trained_model.pt')

# Initialize video capture
cap = cv2.VideoCapture(r'F:\Data Science\Egg detection with count\egg_detection_with_counts\Eggs_On_A_Conveyor_preview.mp4')

# Define reference line coordinates (for counting crossing)
line_position = 700  # X-coordinate of the reference line (adjust based on video)
line_color = (0, 255, 0)  # Green line color
line_thickness = 2

# Initialize variables
object_counter = 0  # To count objects crossing the line
tracked_objects = {}  # To store tracked objects and their positions
object_id = 0  # Unique object ID for tracking

# Create a list to store results
results_list = []

# Function to check if the object has crossed the reference line
def has_crossed_line(prev_x, curr_x, line_x):
    return prev_x < line_x and curr_x >= line_x

start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time() - start_time
    # Perform object detection using YOLOv8
    results = model(frame)

    # Get detected object details
    detections = results[0].boxes.xyxy.cpu().numpy()  # Get bounding box coordinates
    confidences = results[0].boxes.conf.cpu().numpy()  # Get confidence scores

    # Draw the reference line
    cv2.line(frame, (line_position, 0), (line_position, frame.shape[0]), line_color, line_thickness)

    # Track objects and count when they cross the line
    new_tracked_objects = {}

    for i, (x1, y1, x2, y2) in enumerate(detections):
        conf = confidences[i]
        if conf > 0.5:  # Filter based on confidence
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            # Track object positions
            tracked = False
            for obj_id, prev_pos in tracked_objects.items():
                prev_x, prev_y = prev_pos

                # Check if the object is the same by proximity
                if abs(center_x - prev_x) < 50 and abs(center_y - prev_y) < 50:
                    new_tracked_objects[obj_id] = (center_x, center_y)

                    # Check if the object crossed the line
                    if has_crossed_line(prev_x, center_x, line_position):
                        object_counter += 1
                        # Append data to list
                        results_list.append({
                            'Object_ID': obj_id,
                            'Time': current_time,
                            'Bounding_Box': (int(x1), int(y1), int(x2), int(y2)),
                            'Total_Count': object_counter
                        })
                        print(f"Object {obj_id} crossed the line. Total count: {object_counter}")
                    tracked = True
                    break

            # If the object is new, assign it a new ID
            if not tracked:
                object_id += 1
                new_tracked_objects[object_id] = (center_x, center_y)

            # Draw bounding box and center point
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)  # Blue box
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)  # Red center point

    # Update tracked objects for the next frame
    tracked_objects = new_tracked_objects

    # Display the count on the frame
    cv2.putText(frame, f"Count: {object_counter}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the video frame with detection
    cv2.imshow('YOLOv8 Object Counting', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

# Convert list to DataFrame and save to CSV
results_df = pd.DataFrame(results_list)
results_df



0: 384x640 64 eggs, 81.0ms
Speed: 5.0ms preprocess, 81.0ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 62 eggs, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 67 eggs, 18.6ms
Speed: 2.0ms preprocess, 18.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 65 eggs, 18.0ms
Speed: 4.0ms preprocess, 18.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 64 eggs, 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 64 eggs, 19.0ms
Speed: 3.0ms preprocess, 19.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 66 eggs, 14.0ms
Speed: 3.0ms preprocess, 14.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Object 8 crossed the line. Total count: 1

0: 384x640 68 eggs, 14.5ms
Speed: 2.0ms preprocess, 14.5ms inference, 2.0ms postpr

Object_ID       Time          Bounding_Box  Total_Count
0            8   0.857571  (631, 802, 794, 982)            1
1           47   0.962106  (645, 592, 770, 707)            2
2           60   1.030661  (647, 495, 759, 598)            3
3           50   1.163302  (639, 802, 793, 981)            4
4           80   1.260798  (647, 573, 769, 696)            5
..         ...        ...                   ...          ...
106       1237  10.990735  (631, 809, 785, 981)          107
107       1312  11.065265  (649, 679, 780, 816)          108
108       1122  11.103454  (660, 310, 740, 382)          109
109       1279  11.132012  (659, 565, 771, 691)          110
110       1325  11.223120  (656, 433, 750, 527)          111

[111 rows x 4 columns]

In [7]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import time

# Load YOLOv8 model
model = YOLO(r'F:\Data Science\Egg detection with count\egg_detection_with_counts\yolov8m_best_trained_model.pt')

# Initialize video capture
cap = cv2.VideoCapture(r'F:\Data Science\Egg detection with count\egg_detection_with_counts\Eggs_On_A_Conveyor_preview.mp4')

# Define reference line coordinates (for counting crossing)
line_position = 700  # X-coordinate of the reference line (adjust based on video)
line_color = (0, 255, 0)  # Green line color
line_thickness = 2

# Initialize variables
object_counter = 0  # To count objects crossing the line
tracked_objects = {}  # To store tracked objects and their positions
object_id = 0  # Unique object ID for tracking

# Create a list to store results
results_list = []

# Create VideoWriter object to save the video with detections
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
out = cv2.VideoWriter('output_with_detections.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Function to check if the object has crossed the reference line
def has_crossed_line(prev_x, curr_x, line_x):
    return prev_x < line_x and curr_x >= line_x

start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time() - start_time
    # Perform object detection using YOLOv8
    results = model(frame)

    # Get detected object details
    detections = results[0].boxes.xyxy.cpu().numpy()  # Get bounding box coordinates
    confidences = results[0].boxes.conf.cpu().numpy()  # Get confidence scores

    # Draw the reference line
    cv2.line(frame, (line_position, 0), (line_position, frame.shape[0]), line_color, line_thickness)

    # Track objects and count when they cross the line
    new_tracked_objects = {}

    for i, (x1, y1, x2, y2) in enumerate(detections):
        conf = confidences[i]
        if conf > 0.5:  # Filter based on confidence
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            # Track object positions
            tracked = False
            for obj_id, prev_pos in tracked_objects.items():
                prev_x, prev_y = prev_pos

                # Check if the object is the same by proximity
                if abs(center_x - prev_x) < 50 and abs(center_y - prev_y) < 50:
                    new_tracked_objects[obj_id] = (center_x, center_y)

                    # Check if the object crossed the line
                    if has_crossed_line(prev_x, center_x, line_position):
                        object_counter += 1
                        # Append data to list
                        results_list.append({
                            'Object_ID': obj_id,
                            'Time': current_time,
                            'Bounding_Box': (int(x1), int(y1), int(x2), int(y2)),
                            'Total_Count': object_counter
                        })
                        print(f"Object {obj_id} crossed the line. Total count: {object_counter}")
                    tracked = True
                    break

            # If the object is new, assign it a new ID
            if not tracked:
                object_id += 1
                new_tracked_objects[object_id] = (center_x, center_y)

            # Draw bounding box and center point
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)  # Blue box
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)  # Red center point

    # Update tracked objects for the next frame
    tracked_objects = new_tracked_objects

    # Display the count on the frame
    cv2.putText(frame, f"Count: {object_counter}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    # Write the frame with detections to the video file
    out.write(frame)

    # Show the video frame with detection
    cv2.imshow('YOLOv8 Object Counting', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture, video writer, and close windows
cap.release()
out.release()
cv2.destroyAllWindows()

# Convert list to DataFrame and save to CSV
results_df = pd.DataFrame(results_list)
print(results_df)



0: 384x640 102 eggs, 54.0ms
Speed: 4.0ms preprocess, 54.0ms inference, 11.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 93 eggs, 48.0ms
Speed: 5.0ms preprocess, 48.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)
Object 30 crossed the line. Total count: 1

0: 384x640 99 eggs, 47.5ms
Speed: 5.0ms preprocess, 47.5ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
Object 73 crossed the line. Total count: 2

0: 384x640 97 eggs, 48.5ms
Speed: 5.0ms preprocess, 48.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Object 35 crossed the line. Total count: 3

0: 384x640 97 eggs, 47.5ms
Speed: 4.0ms preprocess, 47.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Object 58 crossed the line. Total count: 4

0: 384x640 93 eggs, 46.5ms
Speed: 3.0ms preprocess, 46.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Object 42 crossed the line. Total count: 5

0: 384x640 99 eggs, 46.5ms
Speed

In [9]:
results_df

Object_ID       Time          Bounding_Box  Total_Count
0           30   1.285308  (656, 577, 773, 700)            1
1           73   1.427850  (659, 255, 742, 333)            2
2           35   1.541383  (659, 490, 770, 599)            3
3           58   1.651917  (661, 444, 763, 529)            4
4           42   1.752465  (671, 226, 731, 278)            5
..         ...        ...                   ...          ...
251       1105  30.614091  (644, 677, 782, 818)          252
252       1028  30.737586  (661, 308, 739, 377)          253
253       1101  30.856135  (660, 564, 778, 695)          254
254       1086  31.088298  (662, 496, 774, 600)          255
255       1079  31.200210  (656, 433, 749, 529)          256

[256 rows x 4 columns]